In [1]:
import pandas as pd
import numpy as np

import datetime

import matplotlib.pylab as plt
import matplotlib.dates as mdates
from matplotlib import pyplot
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf

import mysql.connector
from sqlalchemy import create_engine

import ast

In [2]:
DB_CONFIG = {
    "catalog-articles": {
        "host": "catalog-articles-mysql.db00.pro00.eu.idealo.com:3306",
        "user": "readonly",
        "passwd": "readonly",
        "database": "catalog_users"
    },
    "content-statistics": {
        "host": "content-statistics-01.srv00.pro07.eu.idealo.com",
        "user": "content-analyst",
        "passwd": "G8wlmMelLPqC",
        "database": "blackboard"
    },
    "adwords": {
        "host": "adwords-db.db00.pro00.eu.idealo.com",
        "user": "readonly",
        "passwd": "readonly",
        "database": "adwords"
    },
    "catalog-connect": {
        "host": "catalog-connect-mysql.db00.pro00.eu.idealo.com",
        "user": "readonly",
        "passwd": "readonly",
        "database": "catalog_connect"
    },
    
}

# hier give your new table name
#TABLE_NAME = "category_trend_monitoring_FaFa_calender"

def read_db(connection, query):
    try:
        engine = create_engine(f'''mysql+pymysql://{DB_CONFIG[connection]["user"]}:{DB_CONFIG[connection]['passwd']}@{DB_CONFIG[connection]['host']}/{DB_CONFIG[connection]['database']}''')
        data = pd.read_sql(query, engine)
        return data
    except mysql.connector.Error as err:
        print(err)
        #logger.error(err)

def write_db(table_name, connection, data):
    try:
        engine = create_engine(f'''mysql+pymysql://{DB_CONFIG[connection]["user"]}:{DB_CONFIG[connection]['passwd']}@{DB_CONFIG[connection]['host']}/{DB_CONFIG[connection]['database']}''')
        data.to_sql(table_name, engine, if_exists='replace',index=False)
    except mysql.connector.Error as e:
        print(e)
        #logger.info(msg=f"""Unable to save volumes for keyword_id {data['keyword_id'].iloc[-1]}""")
    else:
        print('*** items successfully inserted ***')

In [3]:
sql_query_1 = '''
SELECT DISTINCT attribute_key_id, category_id
FROM category_attribute_key_config
WHERE show_in_main_details = 1    
AND attribute_key_id <> 15583
'''

df_1 = read_db('catalog-connect', sql_query_1)

In [4]:
df_1

,attribute_key_id,category_id
0,16099,18287
1,22716,18287
2,22541,18287
3,26776,18287
4,19060,18287
...,...,...
12711,29221,21135
12712,5070,16272
12713,2500,16272
12714,17532,16272


In [5]:
df_1[df_1.attribute_key_id == 1106]

,attribute_key_id,category_id
2155,1106,30990


In [6]:
attr_list = ["'" + str(i) + "'" for i in df_1.attribute_key_id.tolist()]
str_attr = ', '.join(attr_list)

cat_list = ["'" + str(i) + "'" for i in df_1.category_id.tolist()]
str_cat = ', '.join(cat_list)

In [7]:
sql_query_3 = '''
    SELECT DISTINCT pav.product_id, pav.attribute_key_id, prod.category_id
    FROM product_attribute_value pav 
    INNER JOIN  (SELECT id AS product_id, category_id FROM product) prod
    ON prod.product_id = pav.product_id
    WHERE pav.attribute_key_id IN ('''+ str_attr +''')
    AND prod.category_id IN ('''+ str_cat +''')    
'''

df_t = read_db('catalog-connect', sql_query_3)

In [8]:
df_t1 = df_t.drop('category_id', axis=1)

In [9]:
df_t2 = pd.merge(df_t, df_1, how = 'inner')

In [10]:
df_t2

,product_id,attribute_key_id,category_id
0,3623,21938,19116
1,3757,21938,19116
2,3764,21938,19116
3,3821,21938,19116
4,25186,21938,19116
...,...,...,...
22650194,201698292,31056,32630
22650195,201698294,31056,32630
22650196,201698295,31056,32630
22650197,201698298,31056,32630


In [11]:
df_3 = df_t2.drop_duplicates()

In [12]:
df_3 = df_3.groupby(['product_id', 'category_id']).count().reset_index()

In [13]:
df_3.head()

,product_id,category_id,attribute_key_id
0,3623,19116,1
1,3748,19116,1
2,3757,19116,5
3,3763,19116,1
4,3764,19116,5


In [14]:
df_t2[(df_t2.category_id == 1003)&(df_t2.product_id == 21481)]

,product_id,attribute_key_id,category_id
191494,21481,1014,1003
212450,21481,13586,1003
213051,21481,3961,1003
2326021,21481,20347,1003


In [15]:
sql_query_2 = '''
SELECT product.product_id,
product.category_id,
main_details_count.cat_main_attr_count,
product.variants,
product.parent_id,
product.product_type,
product.online_de,
product.online_at,
product.online_uk,
product.online_fr,
product.online_it,
product.online_es

FROM    (SELECT p.id AS product_id,
     p.category_id,
     p2.variants,
     p.main_product_id as parent_id,
    (CASE
    WHEN p2.variants IS NULL AND p.main_product_id IS NULL THEN 'nonVaried' WHEN p2.variants IS NULL AND p.main_product_id IS NOT NULL THEN 'variant'    WHEN p2.variants > 0 THEN 'mainProduct'    ELSE 'error'    END) AS product_type,
    p.online_de,
    p.online_at,
    p.online_uk,
    p.online_fr,
    p.online_it,
    p.online_es
    FROM       (SELECT id, category_id, main_product_id,
        (CASE WHEN JSON_CONTAINS(online_countries, '"de_DE"') THEN 1 ELSE 0 END) AS online_de,
        (CASE WHEN JSON_CONTAINS(online_countries, '"de_AT"') THEN 1 ELSE 0 END) AS online_at,
        (CASE WHEN JSON_CONTAINS(online_countries, '"en_GB"') THEN 1 ELSE 0 END) AS online_uk,
        (CASE WHEN JSON_CONTAINS(online_countries, '"fr_FR"') THEN 1 ELSE 0 END) AS online_fr,
        (CASE WHEN JSON_CONTAINS(online_countries, '"it_IT"') THEN 1 ELSE 0 END) AS online_it,
        (CASE WHEN JSON_CONTAINS(online_countries, '"es_ES"') THEN 1 ELSE 0 END) AS online_es
        FROM catalog_connect.product
        WHERE online_countries IS NOT NULL) p
    LEFT JOIN   (SELECT main_product_id, (COUNT(id)) AS variants
                FROM catalog_connect.product
                GROUP BY main_product_id) p2
    ON p.id = p2.main_product_id
        ) product
INNER JOIN    (SELECT category_id,
            COUNT(attribute_key_id) AS cat_main_attr_count
            FROM category_attribute_key_config
            WHERE show_in_main_details = 1     AND attribute_key_id <> 15583    
            GROUP BY category_id
            HAVING cat_main_attr_count > 0) main_details_count
ON product.category_id = main_details_count.category_id

'''

df_2 = read_db('catalog-connect', sql_query_2)

In [16]:
df_4 = pd.merge(df_2, df_3, on = ['product_id', 'category_id'], how = 'left')

In [17]:
df_4 = df_4.rename({'attribute_key_id': 'main_count'}, axis=1)

In [18]:
df_4

,product_id,category_id,cat_main_attr_count,variants,parent_id,product_type,online_de,online_at,online_uk,online_fr,online_it,online_es,main_count
0,5108,1003,7,NaN,NaN,nonVaried,0,0,1,0,0,0,2.0
1,5274,1003,7,NaN,NaN,nonVaried,0,0,0,1,1,0,2.0
2,5410,1003,7,NaN,NaN,nonVaried,0,0,0,1,0,0,2.0
3,5438,1003,7,NaN,NaN,nonVaried,1,0,0,0,0,0,2.0
4,11645,1003,7,NaN,NaN,nonVaried,0,0,0,1,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685969,201646323,32668,17,NaN,201646287.0,variant,1,1,1,0,0,1,5.0
3685970,201646324,32668,17,NaN,201646287.0,variant,1,1,0,0,0,0,5.0
3685971,201646326,32668,17,NaN,201646287.0,variant,1,1,0,0,0,0,5.0
3685972,201682675,32668,17,NaN,NaN,nonVaried,1,1,1,1,1,1,4.0


In [19]:
write_db('main_details_products', 'content-statistics', df_4)

*** items successfully inserted ***
